# Imports

In [51]:
import re
import pymongo
import pandas as pd

import sys
sys.path.append(r'../ScrapingTools')
from Image_Processing import CalcImgMetrics
from TTB_scraping import TTB_Scraper

# Connect to database

Existing collections

In [ ]:
# Set up connection to mongodb
client = pymongo.MongoClient() # Connect to default client
db = client.TTB # Get a database (note: lazy evaluation)
TTB = db.TTB # collection for form data
TTBLabels = db.LabelImages # collection for the label image data

Test view

In [25]:
TEST = db.TEST  # test collection

Deleting a table!

In [24]:
# WARNING: deletes database?
#client.drop_database('TEST')

# Pandas into Mongo

### Get some data

In [33]:
TTBID = 16306001000152
scraper = TTB_Scraper(TTBID)  # blue moon, mango wheat
meta, imgs = scraper.get_images()
print(meta)
metric_calculator = CalcImgMetrics(imgs[0])
colors, entropy = metric_calculator.calc_all_metrics()
print(colors)
print(entropy)

[('Label Image: Brand (front)', 'https://www.ttbonline.gov/colasonline/publicViewAttachment.do?filename=BMMangoBtl.jpg&filetype=l'), ('Label Image: Neck', 'https://www.ttbonline.gov/colasonline/publicViewAttachment.do?filename=BMMangoNeck.jpg&filetype=l')]
   percentage           r           g           b
0    0.333333  236.234043  231.394799  212.919622
1    0.083333  194.457666  171.700229  114.830664
2    0.250000   69.646465  160.575758  211.919192
3    0.333333   70.268293   80.341463  104.658537
39852.8310789


/opt/conda/envs/datainc/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [52]:
[re.sub('Label Image: ', '', label) for label in meta[0]]

['Brand (front)',
 'https://www.ttbonline.gov/colasonline/publicViewAttachment.do?filename=BMMangoBtl.jpg&filetype=l']

In [55]:
pd.DataFrame(meta)

,0,1
0,Label Image: Brand (front),https://www.ttbonline.gov/colasonline/publicVi...
1,Label Image: Neck,https://www.ttbonline.gov/colasonline/publicVi...


In [34]:
colors['TTBID'] = [str(TTBID)]* colors.shape[0]

In [35]:
colors

,percentage,r,g,b,TTBID
0,0.333333,236.234043,231.394799,212.919622,16306001000152
1,0.083333,194.457666,171.700229,114.830664,16306001000152
2,0.250000,69.646465,160.575758,211.919192,16306001000152
3,0.333333,70.268293,80.341463,104.658537,16306001000152


### Insert into the database

In [57]:
db.collection_names()

['TEST', 'TTB', 'LabelImages']

In [36]:
#TEST.delete_many({}) # deletes everything in collection

In [37]:
TEST.insert_many(colors.to_dict('records'))

In [38]:
TEST.count()

4

In [39]:
pd.DataFrame(list(TEST.find()))

,TTBID,_id,b,g,percentage,r
0,16306001000152,59dd9c1486c53a0ba3a6b31e,212.919622,231.394799,0.333333,236.234043
1,16306001000152,59dd9c1486c53a0ba3a6b31f,114.830664,171.700229,0.083333,194.457666
2,16306001000152,59dd9c1486c53a0ba3a6b320,211.919192,160.575758,0.250000,69.646465
3,16306001000152,59dd9c1486c53a0ba3a6b321,104.658537,80.341463,0.333333,70.268293
